<a href="https://colab.research.google.com/github/vamshikrishna2001/trial-repo/blob/main/DecodingT5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets torch torchinfo

In [3]:
!pip install torchinfo

In [4]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


# **Imports**

In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW , AutoModelForSeq2SeqLM , AutoTokenizer ,AutoModel
from datasets import load_dataset

# **Loading and Exploring Data**

In [6]:
dataset = load_dataset("knkarthick/dialogsum")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [7]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [8]:
dataset['train'][2]['summary']

"#Person1#'s looking for a set of keys and asks for #Person2#'s help to find them."

# **Loading model**

In [9]:
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [10]:
len(model.decoder.block)

6

In [11]:
dash_line = ''.join('-' for i in range(100))
for i,j in enumerate(model.decoder.block):
  print(dash_line)
  print(i)
  print(j)

----------------------------------------------------------------------------------------------------
0
T5Block(
  (layer): ModuleList(
    (0): T5LayerSelfAttention(
      (SelfAttention): T5Attention(
        (q): Linear(in_features=512, out_features=512, bias=False)
        (k): Linear(in_features=512, out_features=512, bias=False)
        (v): Linear(in_features=512, out_features=512, bias=False)
        (o): Linear(in_features=512, out_features=512, bias=False)
        (relative_attention_bias): Embedding(32, 8)
      )
      (layer_norm): T5LayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): T5LayerCrossAttention(
      (EncDecAttention): T5Attention(
        (q): Linear(in_features=512, out_features=512, bias=False)
        (k): Linear(in_features=512, out_features=512, bias=False)
        (v): Linear(in_features=512, out_features=512, bias=False)
        (o): Linear(in_features=512, out_features=512, bias=False)
      )
      (layer_norm): T5LayerNorm()
   

# **difference between the Model AutoModelForSeq2SeqLM and AutoModel**

**there is a extra layer i.e modelSeq2Seq.lm_head**
# Linear(in_features=512, out_features=32128, bias=False)


In [241]:
modelSeq2Seq = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [242]:
modelSeq2Seq.lm_head

Linear(in_features=512, out_features=32128, bias=False)

# *Start off*

In [14]:
torch.cuda.is_available()

False

In [15]:
for i in model.encoder.parameters():
  break

In [16]:
i.device

device(type='cpu')

In [17]:
i.requires_grad

True

In [18]:
with torch.no_grad():
  for i in model.encoder.parameters():
    break

  print(i.requires_grad)

True


In [19]:
# Check the requires_grad status of parameters before torch.no_grad() context
print("Before torch.no_grad():")
for i in model.encoder.parameters():
    print(i.requires_grad)
    break

with torch.no_grad():
    # Check the requires_grad status of parameters inside torch.no_grad() context
    print("Inside torch.no_grad():")
    for i in model.encoder.parameters():
        print(i.requires_grad)
        break

# Check the requires_grad status of parameters after torch.no_grad() context
print("After torch.no_grad():")
for i in model.encoder.parameters():
    print(i.requires_grad)
    break

Before torch.no_grad():
True
Inside torch.no_grad():
True
After torch.no_grad():
True


In [20]:
import torchinfo
torchinfo.summary(model)

Layer (type:depth-idx)                                  Param #
T5Model                                                 --
├─Embedding: 1-1                                        16,449,536
├─T5Stack: 1-2                                          16,449,536
│    └─Embedding: 2-1                                   (recursive)
│    └─ModuleList: 2-2                                  --
│    │    └─T5Block: 3-1                                3,147,008
│    │    └─T5Block: 3-2                                3,146,752
│    │    └─T5Block: 3-3                                3,146,752
│    │    └─T5Block: 3-4                                3,146,752
│    │    └─T5Block: 3-5                                3,146,752
│    │    └─T5Block: 3-6                                3,146,752
│    └─T5LayerNorm: 2-3                                 512
│    └─Dropout: 2-4                                     --
├─T5Stack: 1-3                                          16,449,536
│    └─Embedding: 2-5             

In [21]:
(93_405_696*4)/(1024*1024*1024)

0.3479633331298828

In [22]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5Model.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
model

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dropout(p=0.1, inplace=

In [243]:
modelSeq2Seq.lm_head

Linear(in_features=512, out_features=32128, bias=False)

In [38]:
lm_head_weights = model.shared.weight.T.clone()
lm_head_weights_detached = lm_head_weights.detach()
lm_head_weights_detached

tensor([[ -2.0156,  12.6250,  -8.7500,  ..., -25.2500, -25.5000, -26.7500],
        [  0.2236,   8.1875,   7.1875,  ..., -28.5000, -29.3750, -28.3750],
        [ -7.0938, -11.6250,  27.8750,  ..., -17.2500, -18.2500, -17.8750],
        ...,
        [ -0.3535,   7.9375, -26.7500,  ..., -17.7500, -17.7500, -18.5000],
        [  2.6406,  -7.3125,   0.8555,  ...,  -5.2500,  -4.8125,  -7.0000],
        [ -2.8906,   0.9453,  -1.5156,  ...,  27.3750,  27.7500,  27.6250]])

In [39]:
lm_head_weights_detached.shape

torch.Size([512, 32128])

In [40]:
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class ModifiedT5Model(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.fc1 = F.Linear(512, 32128,weights=lm_head_weights_detached, bias=False)

    def forward(self, input_ids):
      outputs = self.model(input_ids=input_ids)
      logits = self.fc1(outputs.last_hidden_state)
      return logits


In [244]:
prompt = "Capital of Brazil"

In [245]:
tokenizer.SPECIAL_TOKENS_ATTRIBUTES

['bos_token',
 'eos_token',
 'unk_token',
 'sep_token',
 'pad_token',
 'cls_token',
 'mask_token',
 'additional_special_tokens']

In [246]:
input_ids = tokenizer(prompt,return_tensors='pt').input_ids
tokenizer.convert_ids_to_tokens(list(input_ids.numpy()[0]))

['▁Capital', '▁of', '▁Brazil', '</s>']

In [247]:
input_ids

tensor([[5826,   13, 9278,    1]])

In [253]:
decoder_inputs = tokenizer('<pad>',add_special_tokens=False,return_tensors='pt').input_ids
decoder_inputs

tensor([[0]])

In [251]:
modelSeq2Seq.config.decoder_start_token_id

0

In [252]:
tokenizer.convert_ids_to_tokens([0])

['<pad>']

In [207]:
with torch.no_grad():
  outputs = modelSeq2Seq(input_ids,decoder_input_ids = decoder_inputs, return_dict=True,output_hidden_states=True)

In [254]:
outputs.logits[:, -1:]

tensor([[[-11.6290,  -7.2399, -12.5441,  ..., -40.0261, -40.0633, -40.0102]]])

In [255]:
outputs.logits

tensor([[[-11.6290,  -7.2399, -12.5441,  ..., -40.0261, -40.0633, -40.0102]]])

In [256]:
outputs.logits.shape

torch.Size([1, 1, 32128])

In [257]:
s = [1,2,3]

In [258]:
s[-1:]

[3]

In [259]:
g = outputs.encoder_hidden_states

In [260]:
len(g)

7

In [263]:
next_decoder_input_ids = torch.argmax(outputs.logits,axis = -1)

In [264]:
decoder_inputs

tensor([[0]])

In [265]:
decoder_input_ids = torch.cat([decoder_inputs, next_decoder_input_ids], axis=-1)

In [266]:
tokenizer.convert_ids_to_tokens(next_decoder_input_ids.numpy()[0])

['▁Was']

In [267]:
with torch.no_grad():
  ouputs = modelSeq2Seq.forward(input_ids=None,encoder_outputs=(outputs.encoder_hidden_states,),decoder_input_ids=decoder_input_ids, return_dict=True,output_hidden_states=True)

AttributeError: 'tuple' object has no attribute 'size'

In [236]:
tokenizer.convert_ids_to_tokens(torch.argmax(outputs.logits,axis = -1).numpy()[0])

['▁capital', 'buz']

In [277]:

dash_line = ''.join('-' for i in range(100))
prompt = "what is capital of brazil ?"
input_ids = tokenizer(prompt,return_tensors='pt').input_ids


print(dash_line)
print("prompt in tokens")
print(tokenizer.convert_ids_to_tokens(list(input_ids.numpy()[0])))
decoder_inputs = tokenizer('<pad>',add_special_tokens=False,return_tensors='pt').input_ids
print("decoder_inputs in tokens")
print(tokenizer.convert_ids_to_tokens(list(decoder_inputs.numpy()[0])))
print(dash_line)

with torch.no_grad():
  outputs = modelSeq2Seq(input_ids,decoder_input_ids = decoder_inputs, return_dict=True,output_hidden_states=True)
  next_decoder_input_ids = torch.argmax(outputs.logits,axis = -1)
  print("MODEL OUTPUT ")
  print(dash_line)
  print(tokenizer.convert_ids_to_tokens(next_decoder_input_ids.numpy()[0]))
  print(dash_line)
  decoder_input_ids = torch.cat([decoder_inputs, next_decoder_input_ids], axis=-1)
  print(decoder_input_ids)

  print(dash_line)
  print("encoder_hidden_states")
  g = outputs.encoder_hidden_states
  print(len(g))
  print(g[0].shape)

----------------------------------------------------------------------------------------------------
prompt in tokens
['▁what', '▁is', '▁capital', '▁of', '▁bra', 'zi', 'l', '▁', '?', '</s>']
decoder_inputs in tokens
['<pad>']
----------------------------------------------------------------------------------------------------
MODEL OUTPUT 
----------------------------------------------------------------------------------------------------
['▁Was']
----------------------------------------------------------------------------------------------------
tensor([[   0, 2751]])
----------------------------------------------------------------------------------------------------
encoder_hidden_states
7
torch.Size([1, 10, 512])


In [278]:
for i in range(4):
  with torch.no_grad():
    outputs = modelSeq2Seq.forward(input_ids=None,encoder_outputs=g, decoder_input_ids=decoder_input_ids, return_dict=True,output_hidden_states=True)
    print("MODEL OUTPUT ")
    print(dash_line)
    print(tokenizer.convert_ids_to_tokens(next_decoder_input_ids.numpy()[0]))
    print(dash_line)
    decoder_input_ids = torch.cat([decoder_inputs, next_decoder_input_ids], axis=-1)
    print(decoder_input_ids)

    print(dash_line)
    print("encoder_hidden_states")
    g = outputs.encoder_hidden_states
    print(len(g))
    print(g[0].shape)

MODEL OUTPUT 
----------------------------------------------------------------------------------------------------
['▁Was']
----------------------------------------------------------------------------------------------------
tensor([[   0, 2751]])
----------------------------------------------------------------------------------------------------
encoder_hidden_states
1
torch.Size([10, 512])


ValueError: not enough values to unpack (expected 3, got 2)